In [2]:
import os
import json
import kagglehub

import numpy as np
import pandas as pd
from scipy import sparse
# import tensorflow as tf
# import matplotlib.pyplot as plt

# from tensorflow.keras import Sequential
from kagglehub import KaggleDatasetAdapter
# from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
# from tensorflow.keras.layers import Conv2D, Dropout, BatchNormalization, MaxPooling2D 

c:\Users\devas\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


data preprocessing

In [3]:
title = "arashnic/book-recommendation-dataset"
dataset_path = kagglehub.dataset_download(title)

for dirname, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        print(os.path.join(filename))

Books.csv
classicRec.png
DeepRec.png
Ratings.csv
recsys_taxonomy2.png
Users.csv


In [4]:
dataset_path

'C:\\Users\\devas\\.cache\\kagglehub\\datasets\\arashnic\\book-recommendation-dataset\\versions\\3'

In [4]:
books = pd.read_csv(os.path.join(dataset_path, filenames[0]), delimiter=',')
ratings = pd.read_csv(os.path.join(dataset_path, filenames[3]), delimiter=',')
users = pd.read_csv(os.path.join(dataset_path, filenames[5]), delimiter=',')

C:\Users\devas\AppData\Local\Temp\ipykernel_14252\4279653038.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(os.path.join(dataset_path, filenames[0]), delimiter=',')


In [5]:
min_r = 8
ISBN_list = ratings.loc[ratings['Book-Rating'] >= min_r, 'ISBN']
book_list = books[books['ISBN'].isin(ISBN_list)].copy()[['ISBN', 'Book-Title', 'Year-Of-Publication']]
book_list = book_list[:int(book_list.shape[0]*0.4)].copy()
book_list = book_list.reset_index(drop=True)

train_ratings, test_ratings = train_test_split(ratings[ratings['Book-Rating'] >= min_r], test_size=0.2, random_state=42)

In [6]:
books.columns, ratings.columns, users.columns, book_list.columns, test_ratings.columns

(Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
        'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
       dtype='object'),
 Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object'),
 Index(['User-ID', 'Location', 'Age'], dtype='object'),
 Index(['ISBN', 'Book-Title', 'Year-Of-Publication'], dtype='object'),
 Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object'))

model training

In [7]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(book_list['Book-Title'] + ' ' + book_list['Year-Of-Publication'].astype(str))
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
max_rec = 10

In [8]:
def rec_func(isbn_user, book_list=book_list, cosine_sim=cosine_sim, max_rec=max_rec):
    isbn = book_list.index[book_list['ISBN'] == isbn_user].to_list()[0]
    sim_score = list(enumerate(cosine_sim[isbn]))
    sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)[1:max_rec+1]
    idxs = [i[0] for i in sim_score]

    return book_list.iloc[idxs][['ISBN', 'Book-Title', 'Year-Of-Publication']]

In [9]:
example_isbn = book_list.iloc[0]['ISBN']
recs = rec_func(example_isbn)

evaluation

In [10]:
def precision_at_k(test=test_ratings, book_list=book_list, max_rec=max_rec):
    precision =[]
    precision =[]
    user_ids = [user_id for user_id in test['User-ID'].unique()]
    rels = [test[test['User-ID'] == user_id]['ISBN'].values for user_id in user_ids]

    sel_isbns = []
    for idx_arr in rels:
        mask = np.isin(idx_arr, book_list['ISBN'].values)
        if mask.any():
            for idx in idx_arr[mask]:
                sel_isbns.append(idx)
    
    pred_recs = [isbn for sel_isbn in sel_isbns for isbn in rec_func(sel_isbn)['ISBN'].values]

    hits = len(set(pred_recs) & set(sel_isbns))
    
    precision.append(hits / len(pred_recs))
    return np.mean(precision) if precision else 0.0

In [ ]:
pr = precision_at_k()

In [ ]:
print(f"precision: {(pr * 100):2f} / 100")

precision: 4.225660


deployment

In [ ]:
sparse.save_npz('tfidf_matrix.npz', tfidf_matrix)

In [ ]:
np.save('cosine_sim.npy', cosine_sim)